# Optimisation and its Pitfalls

In the context of PKPD modelling numerical optimisation is often used to find estimates for model parameters by minimising an objective function $L$ that quantifies the distance of the model predictions to the measured data. In this notebook we discuss some of the common reasons why naïve application of numerical optimisation may fail to find the best parameter set, and more importantly may fail to consistently produce estimates over multiple optimisation runs.

This notebook is complementary to the identifiability notebooks provided in this repository [1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb).

## Numerical optimisation

Numerical optimisation refers to the problem of finding a parameter set $\hat \psi $ in a defined search space for some objective function $L$ that minimises its value, i.e. $L(\hat \psi) \leq L(\psi ), \forall \psi \in \mathcal{S}$, where $\mathcal{S}$ is the search space of all allowed parameter values. In the context of PKPD modelling optimisation may address the question of finding the model parameter set $\hat \psi $ that best describes the observed dose-response of a patient for a given PKPD model. This parameter set can then be used to predict the behaviour of different, experimentally untested treatment strategies, and in that way optimise the treatment for that patient. Here, the objective function quantifies how far off the model predictions are from the measured values. As a result, finding the parameter set $\hat \psi $ in $\mathcal{S}$ that minimises the objective function $L$, is equivalent to finding the parameter set that makes the model predictions closest to the observations. Under the assumption that the model is suited to describe the pharmacokinetics and pharmacodynamics of the drug, one might then argue that the those parameters capture the PKPD of the individual, and are therefore suited to predict the dose-response for untested dosing strategies.

A major challenge in numerical optimisation is, however, that optimisers might fail to find the best parameter set for a patient, because either the model is not identifiable, or the numerical optimiser itself is not able to explore the search space $\mathcal{S}$ due to numerical limitations. In this notebook, we explore the limitations of numerical optimisation that we encountered in this project. We discuss in detail how we overcame / avoided those problems in the hope that they may generalise to other PKPD studies. The challenges involved with model non-identifiabilty are discussed in a separate notebook, [Identifying Structural and Practical Non-Identifiability](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/inference_pitfalls/identifiability.ipynb).

Note that we advocate to use Bayesian inference for parameter inference to not only estimate the optimal parameter set for a patient, but also quantify the certainty the data provides in those estimates. As a result, the model parameters are not actually found by optimising an objective function, but by sampling from a posterior distribution. However, the numerical challenges are very similar for both methods, and conceptually optimisation of an objective function is easier to understand. That is why we chose to explore optimisation in this notebook first. For a similar analysis of the pitfalls of sampling, please have a look at the dedicated notebook, [MCMC Sampling and its Pitfalls].

## Toy problem: Tumour growth in absence of treatment

For clarity let us illustrate the pitfalls of numerical optimisers for a specific example. In [n1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb) we introduced the structural model reported in [1] for the tumour growth in absence of treatment

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0, \lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1}.
\end{equation*}

Here, $\lambda _0$ is the exponential growth rate, and $\lambda _0$ is the linear growth rate. We can see that for small tumour tumour volumes $V^s_T\ll V_{\text{crit}}$ the rate of change of the tumour volume is proportional to itself (exponential growth), and for large tumour volumes $V^s_T\gg V_{\text{crit}}$ the rate of change assumes a constant growth rate (linear growth). The critical volume can be expressed in terms of the ratio of the growth rates $V_{\text{crit}} = \lambda _1 / 2 \lambda _0$.

In order to keep the toy problem as simple as possible, and to be able to know when the optimisation was successful, we do not try to find an optimal parameter set for real data, but instead choose to generate artificial data with some known parameter set

\begin{equation*}
    \psi ^* = (V^*_0, \lambda ^*_0, \lambda ^*_1) = (0.14\, \text{cm}^3, 0.28\, 1/\text{day}, 0.13\, \text{cm}^3/\text{day}).
\end{equation*}

This parameter set is close to one of the estimates found in [n1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb), such that a forward simulation of the tumour growth model over a time frame of $30 \text{day}$ closely resembles the experimental data. However, note that for simplicity we do not introduce noise to the simulated observations at first.

The toy inverse problem is completed by defining the Squared Distance Error Measure as our objective function

\begin{equation*}
    L(\psi | V^\text{obs}_{T}) = \sum ^{n}_{i=1}\left( V^\text{obs}_{T, i} - V^s_T(t_i, \psi)\right) ^2,
\end{equation*}

where $n$ is the number of simulated measurements, $V^\text{obs}_{T, i}$ is the "measured" tumour volume at time $t_i$ and $V^s_T(t_i, \psi)$ is the model prediction at time $t_i$ for model parameters $\psi $. This objective function is greater or equal to zero for all $\psi $, and only vanishes for the data generating parameter set $\psi ^*$ for which the model identically reproduces the observations.

Note: This model does not suffer from structural nor practical non-identifiabilities when the data is generated with $\psi ^*$ and the tumour volumes are "measured" twice a week for a period of 30 days. As a result, this model is ideally suited to explore some of the generic limitations of numerical optimisation. 

## Generating the data

We now generate the "measured" tumour volumes by solving the above defined ordinary differential equation for the tumour growth in absence of treatment with the parameter set $\psi ^*$. We define the "measurement" time points of the tumour volume by the real measurement time points from mouse ID 40 in the LXF A677 control growth dataset (roughly twice a week for 30 days), see [n2](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/data_preparation.ipynb) for details.

In [2]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as m


# Wrap myokit model, so it can be used with pints
class PintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here V_0, lambda_0, lambda_1.
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, lambda_0, lambda_1 = parameters

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.lambda_0', lambda_0)
        self.sim.set_constant('central.lambda_1', lambda_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [3]:
#
# Simulate "noise-free" data for with V_0 = 0.14 cm^3, lambda_0 = 0.28 1/day, lambda_1= 0.13 cm^3/day. 
#
# This cell needs the above defined model.
# [PintsModel]
#

import os

import numpy as np
import pandas as pd


# Define true parameters
true_initial_volume = 0.14  # in cm^3
true_lambda_0 = 0.28  # in 1/day
true_lambda_1 = 0.13  # in cm^3/day
true_parameters = np.array([true_initial_volume, true_lambda_0, true_lambda_1])

# Define simulation times in day from dataset
# Get path to directory
path = os.path.dirname(os.getcwd())
path = path + '/lung_cancer/control_growth/data/lxf_control_growth.csv'

# Get time points from dataset
data = pd.read_csv(path)
mask_mouse = data['#ID'] == 40
mask_nans = ~data[mask_mouse]['TIME in day'].isnull()
data_times = data[mask_mouse]['TIME in day'][mask_nans].to_numpy()
del data

# Instantiate model
model = PintsModel()

# Simulate data
data_volumes = model.simulate(true_parameters, data_times)

# Create simulated dataset
# Shape (2, n_times)
data = np.vstack((data_times, data_volumes))


In [33]:
#
# Visualise the simulated dataset.
#
# This cell needs the above generated dataset:
# [data]
#

import plotly.graph_objects as go


# Create figure
fig = go.Figure()

# Get time points
times = data[0, :]

# Get observed tumour volumes
observed_volumes = data[1, :]

# Plot tumour volume over time
fig.add_trace(go.Scatter(
    x=times,
    y=observed_volumes,
    name="Simulated tumour growth",
    showlegend=True,
    hovertemplate=
        "<b>Simulated tumour growth</b><br>" +
        "Time: %{x:} day<br>" +
        "Tumour volume: %{y:.02f} cm^3<br>" +
        "<extra></extra>",
    mode="markers",
    marker=dict(
        symbol='circle',
        opacity=0.7,
        line=dict(color='black', width=1))
))

# Set X, Y axis and figure size
fig.update_layout(
    autosize=True,
    xaxis_title=r'$\text{Time in day}$',
    yaxis_title=r'$\text{Tumour volume in cm}^3$',
    template="plotly_white")

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()

**Figure 1:** Generated tumour growth data. The tumour growth model in absence of treatment was solved for model parameters $(V^*_0, \lambda ^*_0, \lambda ^*_1) = (0.14\, \text{cm}^3, 0.28\, 1/\text{day}, 0.13\, \text{cm}^3/\text{day})$ and tumour volumes were recorded according to the measurement times of mouse ID 40 in the LXF A677 control growth dataset. Note that no noise was added to the "measured" tumour volumes.

## Naïve optimisation

In [n1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb) we have seen that naïve optimisation of the squared distance between the observed tumour volume and the predicted tumour volume did not lead to robust parameter estimates, when we ran the CMAES optimiser multiple times with random initial starting points in $[10^{-3}, 10^3]$ for all parameters. We start our analysis by reproducing this result on the generated dataset.

In [9]:
#
# Run naïve optimisation multiple times, randomly initialised in [10^-3, 10^3].
#
# This cell needs the above defined wrapped myokit model and the simulated data:
# [PintsModel, data]
#

import myokit
import numpy as np
import pints


# Define number of optimisation runs for each individual
n_runs = 10

# Define container for the parameter estimates
# Shape (n_opt_types, n_runs, n_parameters)
n_parameters = 3
naive_estimates = np.empty(shape=(n_runs, n_parameters))

# Define container for the objective function score
naive_estimates_scores = np.empty(shape=n_runs)

# Define random starting points over many orders of magnitude
# Shape = (n_runs, n_parameters)
initial_parameters = np.random.uniform(low=1E-3, high=1E3, size=(n_runs, n_parameters))

# Get relevant time points
times = data[0, :]

# Get measured tumour volumes
observed_volumes = data[1, :]

# Create inverse problem
problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

# Create sum of squares error objective function
error = pints.SumOfSquaresError(problem)

# Run optimisation multiple times
for run_id, initial_params in enumerate(initial_parameters):
    # Create optimisation controller with a CMA-ES optimiser
    optimiser = pints.OptimisationController(
        function=error,
        x0=initial_params,
        method=pints.CMAES)

    # Disable logging mode
    optimiser.set_log_to_screen(False)

    # Parallelise optimisation
    optimiser.set_parallel(True)

    # Set maximal number of iterations
    optimiser.set_max_iterations(iterations=500)

    # Find optimal parameters
    try:
        estimates, score = optimiser.run()
    except:
        # If inference breaks fill estimates with nan
        estimates = np.array([np.nan, np.nan, np.nan])
        score = np.nan

    # Save estimates and score (back transformed to linear scale)
    naive_estimates[run_id, :] = estimates
    naive_estimates_scores[run_id] = score

In [16]:
#
# Visualisation of the spread of naively optimised model parameters for multiple runs from different initial points.
#
# This cell need the above obtain naive parameter estimates, their scores and the generating parameter values.
# [naive_estimates, naive_estimates_scores, true_parameters]
#

import plotly.graph_objects as go


# Get optimised parameters
estimates = naive_estimates

# Get optimised parameters
scores = naive_estimates_scores

# Create figure
fig = go.Figure()

# Create box plot of for initial tumour volume
fig.add_trace(
    go.Box(
        y=estimates[:, 0],  
        name='Initial tumour volume in cm^3',
        boxpoints='all',
        jitter=0.2,
        pointpos=-1.5,
        visible=True,
        marker=dict(
            symbol='circle',
            opacity=0.7,
            line=dict(color='black', width=1)),
    ))

# Create box plot for lambda_0
fig.add_trace(
    go.Box(
        y=estimates[:, 1],  
        name='Exponential growth rate in 1/day',
        boxpoints='all',
        jitter=0.2,
        pointpos=-1.5,
        visible=True,
        marker=dict(
            symbol='circle',
            opacity=0.7,
            line=dict(color='black', width=1))
    ))

# Create box plot of for exponential tumour growth
fig.add_trace(
    go.Box(
        y=estimates[:, 2],  
        name='Linear growth rate in cm^3/day',
        boxpoints='all',
        jitter=0.2,
        pointpos=-1.5,
        visible=True,
        marker=dict(
            symbol='circle',
            opacity=0.7,
            line=dict(color='black', width=1)),
    ))
    
# Create box plot of for objective function score
fig.add_trace(
    go.Box(
        y=scores,  
        name='Objective function score',
        boxpoints='all',
        jitter=0.2,
        pointpos=-1.5,
        visible=True,
        marker=dict(
            symbol='circle',
            opacity=0.7,
            line=dict(color='black', width=1)),
    ))

# Plot true parameters
fig.add_trace(
    go.Scatter(
        x=[
            'Initial tumour volume in cm^3',
            'Exponential growth rate in 1/day',
            'Linear growth rate in cm^3/day'],
        y=true_parameters,
        name='True parameters',
        hovertemplate=
            "<b>True parameters </b><br>" +
            "Initial tumour volume: %.02f in cm^3<br>" % true_parameters[0] +
            "Exponential growth rate: %.02f in 1/day<br>" % true_parameters[1] +
            "Linear growth rate: %.02f in cm^3/day<br>" % true_parameters[2] +
            "<extra></extra>",
        visible=True,
        marker=dict(
            symbol='star',
            opacity=0.7,
            line=dict(color='black', width=1),
            color='gold'),
        mode="markers",
    ))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white",
    yaxis_title='Parameter value')

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


**Figure 2:** Distribution of parameter estimates after naïve optimisation. We randomly initialised the starting points of the optimisations in $[10^{-3}, 10^3]$ for all parameters, and ran the CMAES optimiser for 500 iterations. 

Figure 2 shows that naïve application of the CMAES optimiser cannot reliably recover the generating model parameters within a single run. It is, however, interesting to note that the median of 10 runs seems to robustly recover the true model parameters. 

## Toy model: Tumour growth in absence of treatment

For clarity let us illustrate the pitfalls of numerical optimisers for a specific example. In [1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb) we introduced a structural model for the tumour growth in absence of treatment in which the tumour grows exponentially for tumour volumes $V^s_T$ below a critical volume $V_{\text{crit}}$, and linearly above the critical tumour volume

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} = \frac{\lambda V^s_T}{V^s_T / V_{\text{crit}} + 1}.
\end{equation*}

Here, $\lambda $ is the exponential growth rate. We can see that for large tumour volumes $V^s_T\gg V_{\text{crit}}$ the rate of change of the tumour will assume a constant growth rate $\lambda V_{\text{crit}}$, i.e. the tumour grows linearly for large tumour volumes.

We generate a dataset by forward simulating the model for the structural model parameters

\begin{equation*}
    \psi = (V_0, V_{\text{crit}}, \lambda ) = (0.14\, \text{cm}^3, 0.24\, \text{cm}^3, 0.55\, 1/\text{day}),
\end{equation*}

which we found in [1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb) for mouse ID 40. We will record the tumour volume noise-free every second day for a period of 30 days to generate a dataset similar to the experimental observations.

The toy inverse problem is completed by defining the Squared Distance Error Measure as our objective function

\begin{equation*}
    L(\psi | V^\text{obs}_{T}) = \sum ^{n}_{i=1}\left( V^\text{obs}_{T, i} - V^s_T(t_i, \psi)\right) ^2,
\end{equation*}

where $n$ is the number of simulated measurements, $V^\text{obs}_{T, i}$ is the "measured" tumour volume at time $t_i$ and $V^s_T(t_i, \psi)$ is the model prediction at time $t_i$ for model parameters $\psi $.

## 1. Limiting the search space

A generic problem of numerical optimisation routines is that there is only a finite window of parameter values that can be feasibly explored. Outside this window evaluation of the objective function will have significant numerical erros.

Let us begin by showing objecive function over range of many parameters:

We use dimensionless model with log-transformed parameters as in 1.

$v_0 = 0.14$, $a_0 = 0.24$, $a_1 = 0.24 * 0.55$

In [1]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as m


# Wrap myokit model, so it can be used with pints
class PintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_dimless_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here log v, log a_0, log a_1.
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, log_parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, a_0, a_1 = np.exp(log_parameters)

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.a_0', a_0)
        self.sim.set_constant('central.a_1', a_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [2]:
#
# Simulate "noise-free" data for with v_0 = 0.14, a_0 = 0.24, a_1= 0.24 * 0.55. 
#
# This cell needs the above defined model.
# [PintsModel]
#

import numpy as np


# Define characteristic scales
characteristic_volume = 1  # in cm^3
characteristic_time = 1  # in day

# Define simulation times in day
data_times = np.linspace(start=0, stop=30, num=15)

# Define model parameters v_0, a_0, a_1
true_log_parameters = np.log([0.14, 0.24, 0.24 * 0.55])

# Instantiate model
model = PintsModel()

# Define dimensionless times
dimless_times = data_times / characteristic_time

# Simulate data
dimless_volumes = model.simulate(true_log_parameters, dimless_times)
data_volumes = dimless_volumes * characteristic_volume


In [19]:
#
# Run optimisation multiple times with and without boundaries
#
# This cell needs the above defined wrapped myokit model and the simulated data, as well as the characteristic time and volume scale:
# [PintsModel, simulated_data, characteristic_time, characteristic_volume]
#

import myokit
import numpy as np
import pandas as pd
import pints


# Define number of different optimisation settings
# [bounded search space, open search space]
n_opt_types = 2

# Define number of optimisation runs for each individual
n_runs = 10

# Define container for the structural model estimates
# Shape (n_opt_types, n_runs, n_parameters)
n_parameters = 3
recovered_parameters = np.empty(shape=(n_opt_types, n_runs, n_parameters))

# Define container for the objective function score for the optimised parameters
recovered_parameters_scores = np.empty(shape=(n_opt_types, n_runs))

# Define random starting points over many orders of magnitude
# Shape = (n_runs, n_parameters)
initial_parameters = np.random.uniform(low=1E-3, high=1E3, size=(n_runs, n_parameters))

# Find mouse parameters
for index in range(n_opt_types):
    # Get relevant time points
    times = data_times / characteristic_time

    # Get measured tumour volumes
    observed_volumes = data_volumes / characteristic_volume

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create sum of squares error objective function
    error = pints.SumOfSquaresError(problem)

    # Create boundaries to biologically relevant values
    boundary = pints.RectangularBoundaries(lower=np.log([1E-3]*3), upper=np.log([1E3]*3))

    # Run optimisation multiple times
    for run_id, initial_params in enumerate(initial_parameters):
        # Transform parameters to log-scale
        log_initial_params = np.log(initial_params)

        # Create optimisation controller with a CMA-ES optimiser
        if index == 0:
            # Create optimiser with bounded search space
            optimiser = pints.OptimisationController(
                function=error,
                x0=log_initial_params,
                boundaries=boundary,
                method=pints.CMAES)
        if index == 1:
            # Create optimiser with bounded search space
            optimiser = pints.OptimisationController(
                function=error,
                x0=log_initial_params,
                method=pints.CMAES)

        # Disable logging mode
        optimiser.set_log_to_screen(False)

        # Parallelise optimisation
        optimiser.set_parallel(True)

        # Find optimal parameters
        try:
            estimates, score = optimiser.run()
        except:
            # If inference breaks fill estimates with nan
            estimates = np.array([np.nan, np.nan, np.nan])
            score = np.nan

        # Save estimates and score (back transformed to linear scale)
        recovered_parameters[index, run_id, :] = np.exp(estimates)
        recovered_parameters_scores[index, run_id] = score


----------------------------------------
Unexpected termination.
Current best score: 10.156246398491989
Current best position:
 4.51990710707117271e-01
 6.82247804558307848e+02
 6.78276907497045613e+02
----------------------------------------

----------------------------------------
Unexpected termination.
Current best score: 2.607533564388874
Current best position:
-9.46999195455406062e-02
 5.96300708013394569e+02
 5.93069773285688484e+02
----------------------------------------


In [20]:
#
# Transform dimensionless parameters to new set of biological parameters.
# 
# This cell needs the above inferred dimensionless parameters and the characteristic volume and time scale:
# [recovered_parameters, characteristic_volume, characteristic_time]
#

import numpy as np


# Initialise container for backtransformed paramters
# Shape (n_mice, n_runs, n_parameters)
parameters = np.empty(shape=recovered_parameters.shape)

# Transform initial volumes
parameters[:, :, 0] = recovered_parameters[:, :, 0] * characteristic_volume

# Transform critical volume 
# V_crit = a_0 * V^c
parameters[:, :, 1] = recovered_parameters[:, :, 1] * characteristic_volume

# Transform exponential growth rate
# lambda = a_1 / a_0 / t^c
parameters[:, :, 2] = \
    recovered_parameters[:, :, 2] / recovered_parameters[:, :, 1] / characteristic_time


In [55]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised initial parameter from psi_0=(1, 1, 1) and the five runs from a random initial starting point, and their respective objective function scores, as well as the data
# [parameters, recovered_parameters_scores]
#

import plotly.colors
import plotly.graph_objects as go


# Get number of optimisation types
n_opt_types = parameters.shape[0]

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_opt_types]

# Get optimised parameters
optimised_parameters = parameters

# Get optimised parameters
scores = recovered_parameters_scores

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index in range(n_opt_types):
    # Get optimised parameters
    params = optimised_parameters[index, ...]

    # Get scores
    score = scores[index, :]

    # Label
    label = "Limited search space" if index == 0 else "Unlimited search space"

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=params[:, 0],  
            name=label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))

    # Create box plot for critical volume
    fig.add_trace(
        go.Box(
            y=params[:, 1],  
            name=label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=params[:, 2],  
            name=label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Score: " + label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[
                        {"visible": [True, False, False, True]*2},
                        {"yaxis": {"title": r'$\text{Initial tumour volume in cm}^3$'}}],
                    label="Initial tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, True, False, True]*2},
                        {"yaxis": {"title": r'$\text{Critical tumour volume in cm}^3$'}}],
                    label="Critical tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, False, True, True]*2},
                        {"yaxis": {"title": r'$\text{Growth rate in 1/day}$'}}],
                    label="Growth rate",
                    method="update"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


**Figure 1:** Limited versus unlimited search space. Initialised over $[10^{-3}, 10^3]$

Let us now try to understand why the optimisation fails by looking at the objective function over a significant parameter range

In [89]:
#
# Evaluate objective function over many orders of magnitude.
#
# This cell needs the above simulated data and the generating set of model parameters
# [data_times, data_volumes]
#

#
# Run optimisation multiple times with and without boundaries
#
# This cell needs the above defined wrapped myokit model and the simulated data, as well as the characteristic time and volume scale:
# [PintsModel, simulated_data, characteristic_time, characteristic_volume, true_log_parameters]
#

import myokit
import numpy as np
import pints


# Define range of model parameters (V_0, V_crit, lambda)
initial_volume_range = np.logspace(start=-4, stop=4)
critical_volume_range = np.logspace(start=-4, stop=4)
growth_rate_range = np.logspace(start=-4, stop=4)

# Collect parameter ranges for illustration (next cell)
parameter_range = np.vstack((
    initial_volume_range,
    critical_volume_range,
    growth_rate_range))

# Convert to dimensionless parameters for simulation (log v_0, log a_0, log a_1)
# Shape (n_parameters, n_evaluations)
log_parameter_range = np.vstack((
    np.log(initial_volume_range / characteristic_volume), 
    np.log(critical_volume_range / characteristic_volume),
    np.log(growth_rate_range * 0.24 * characteristic_time / characteristic_volume)))
n_parameters, n_evaluations = log_parameter_range.shape

# Define container for objective scores
# Shape (n_parameters, n_evaluations)
range_scores = np.empty(shape=(n_parameters, n_evaluations))

# Convert data times to dimensionless time
times = data_times / characteristic_time

# Convert data volumes to dimensionless volumes
observed_volumes = data_volumes / characteristic_volume

# Create inverse problem
problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

# Create sum of squares error objective function
error = pints.SumOfSquaresError(problem)

for parameter_id in range(n_parameters):
    # Intialise parameters in true parameters (v_0, a_0, a_1)
    log_params = true_log_parameters.copy()

    for eval_id in range(n_evaluations):
        # Update parameters
        log_params[parameter_id] = log_parameter_range[parameter_id, eval_id]

        try:
            range_scores[parameter_id, eval_id] = error(log_params)
        except myokit.SimulationError:
            range_scores[parameter_id, eval_id] = np.nan

In [105]:
#
# Visualise objective over range of model parameters.
#
# This cell needs above explored range of objective function scores:
# [range_scores, parameter_range]
#

import plotly.graph_objects as go


# Define parameter names
parameter_names = [
    'Initial tumour volume',
    'Critical tumour volume',
    'Growth rate']

# Create figure
fig = go.Figure()

# Line plots of objective function scores
for index, scores in enumerate(range_scores):
    # Get parameters
    params = parameter_range[index, :]
    
    # 
    label

    # Scores over range of parameter values
    fig.add_trace(go.Scatter(
        x=params,
        y=scores,
        name=parameter_names[index],
        showlegend=True,
        visible=True if index == 0 else False,
        hovertemplate=
            "<b>Varying %s:</b><br>" % parameter_names[index] +
            "Score: %{y:.02f}<br>" +
            "Parameter value: %{x:.02f}<br>" +
            "True parameters: V_0 = 0.14, V_crit = 0.24, lambda = 0.55<br>" +
            "<extra></extra>",
        mode="lines"))

# Set X, Y axis and figure size
fig.update_layout(
    autosize=True,
    xaxis_title=r'$\text{Initial tumour volume in cm}^3$',
    yaxis_title=r'$\text{Objective function score in dimensionless}$',
    xaxis_type='log',
    template="plotly_white")

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        dict(
            type = "buttons",
            direction = "down",
            buttons=list([
                dict(
                    args=[
                        {"visible": [True, False, False]},
                        {"xaxis.title": r'$\text{Initial tumour volume in cm}^3$'}],
                    label="Initial tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, True, False]}, 
                        {"xaxis.title": r'$\text{Critical tumour volume in cm}^3$'}],
                    label="Critical tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, False, True]}, 
                        {"xaxis.title": r'$\text{Growth rate in 1/day}$'}],
                    label="Growth rate",
                    method="update"
                ),
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=1.07,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()

**Figure 2:** Error scores when varying each parameter at ones. Everything looks fine if we stay reasonably close to the true solution. We can see already that the objective function score becomes very large

In [117]:
recovered_parameters_scores

array([[4.57036993e-22, 1.03055765e-22, 2.81387582e-23, 3.72033470e-24,
        4.32167027e-22, 3.97276543e-23, 4.39952606e-22, 4.30252877e-22,
        4.45667717e-22, 1.52901886e-22],
       [7.57886358e-02,            nan, 4.31347107e-22, 6.52685148e-01,
        4.32221700e-22,            nan, 4.34219163e-22, 4.31641708e-22,
        6.52685148e-01, 5.02386775e-22]])

In [120]:
recovered_parameters[1]

array([[1.05428730e-019, 1.31133830e-225, 1.07855127e-001],
       [            nan,             nan,             nan],
       [1.40000000e-001, 2.40000000e-001, 1.32000000e-001],
       [5.24897534e-001, 2.14015001e+152, 1.37831284e+151],
       [1.40000000e-001, 2.40000000e-001, 1.32000000e-001],
       [            nan,             nan,             nan],
       [1.40000000e-001, 2.40000000e-001, 1.32000000e-001],
       [1.40000000e-001, 2.40000000e-001, 1.32000000e-001],
       [5.24897560e-001, 8.00111012e+009, 5.15292507e+008],
       [1.40000000e-001, 2.40000000e-001, 1.32000000e-001]])

Model becomes non-identifiable for large critical volumes: score is no longer sensitive to changes in the critical volume and model reduces to 

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} \approx \lambda V_{\text{crit}} V^s_T.
\end{equation*}

So restricting the search space is really just preventing non-identifiabilities?

Let us explore this in detail by fixing the initial tumour volume to the true tumour volume and running an optimisation

## Why does rewriting structural model help optimisation?

Before

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0\lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1}.
\end{equation*}

This model does not suffer from non-identifiabilities in any limit.

## Notebook references

- <a name="ref1"> [n1] </a> [Identifiability of the Structural Model for Lung Cancer Growth in Absence of Treatment](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb)
- <a name="ref2"> [n2] </a> [Lung Cancer Tumour Growth in Absence of Treatment](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/data_preparation.ipynb)

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)